# Building a Model of Tanzania well quality

In [87]:
#Import packages and settings
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


import catboost
import time

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from catboost import Pool, sum_models
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

%matplotlib inline

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [ ]:
FOLDS = 5
SEEDS = [0, 42, 888, 1042, 8888]
VERSION = round(time.time())


In [89]:
df.head()
df.dtypes

id                                   int64
status_group                        object
amount_tsh                         float64
date_recorded                       object
funder                              object
gps_height_x                         int64
installer                           object
longitude_x                        float64
latitude_x                         float64
wpt_name                            object
num_private                          int64
basin                               object
subvillage                          object
region                              object
region_code                          int64
district_code                        int64
lga                                 object
ward                                object
population                           int64
public_meeting                      object
recorded_by                         object
scheme_management                   object
scheme_name                         object
permit     

In [90]:
#Load the three datasets I was provided
df_labels = pd.read_csv("training_set_labels.csv")
df_train_set = pd.read_csv("training_set_values.csv")
df_test = pd.read_csv('test_set_values.csv')

#Merge Labels / Value dataset into our core dataframe
df = pd.merge(df_labels,df_train_set,how="inner",on="id")

In [91]:
def set_classifier(df):
        df['status_group']=df['status_group'].astype('object')

### Long/Lat and GPS_Height data can be approximated using a priority waterfall of average Regional / district code / ward / lga location data. To do this we calculate average location data for non-zero values for our initial dataset

In [92]:
#get region / ward / lga averages for gps_height 
height_fdf = df[df['gps_height'] !=0]

#group by location granularity
height_region_fdf = height_fdf.groupby('region')
height_ward_fdf = height_fdf.groupby('ward')
height_district_code_fdf = height_fdf.groupby('district_code')
height_lga_fdf = height_fdf.groupby('lga')

#calculate average for the region
height_avg_by_region = height_region_fdf.agg({'gps_height': 'mean'},index = True)
height_avg_by_ward = height_ward_fdf.agg({'gps_height': 'mean'})
height_avg_by_district_code = height_district_code_fdf.agg({'gps_height': 'mean'})
height_avg_by_lga = height_lga_fdf.agg({'gps_height': 'mean'})

#some regions had no gps_height data, thus region values were looked up on google and appended
height_region_additional = pd.DataFrame({"region":["Kagera","Tabora","Dodoma","Mbeya","Pwani"],"gps_height":[786,1200,1120,1758,188]})
#concating the average height tables to create a single regional height lookup
height_avg_by_region = height_avg_by_region.reset_index()
height_avg_by_region_complete = pd.concat([height_avg_by_region, height_region_additional], ignore_index=True)

In [93]:
#get region / ward / lga averages for lat / long
#create table with longitude values (note if longitude didnt exist then latitude didn't either)
longitude_fdf = df[df['longitude'] !=0]

#group by granularity
longitude_region_fdf = longitude_fdf.groupby('region')
longitude_ward_fdf = longitude_fdf.groupby('ward')
longitude_district_code_fdf = longitude_fdf.groupby('district_code')
longitude_lga_fdf = longitude_fdf.groupby('lga')

#calculate average for the region
longitude_avg_by_region = longitude_region_fdf.agg({'longitude': 'mean','latitude': 'mean'})
longitude_avg_by_ward = longitude_ward_fdf.agg({'longitude': 'mean','latitude': 'mean'})
longitude_avg_by_district_code = longitude_district_code_fdf.agg({'longitude': 'mean','latitude': 'mean'})
longitude_avg_by_lga = longitude_lga_fdf.agg({'longitude': 'mean','latitude': 'mean'})

In [94]:
#add average Lat/Long to Dataframe
df = pd.merge(df,longitude_avg_by_region,how="left",on="region").rename(columns={'longitude_y':'average_region_longitude','latitude_y':'average_region_latitude'})
df = pd.merge(df,longitude_avg_by_district_code,how="left",on="district_code").rename(columns={'longitude':'average_district_code_longitude','latitude':'average_district_code_latitude'})
df = pd.merge(df,longitude_avg_by_ward,how="left",on="ward").rename(columns={'longitude':'average_ward_longitude','latitude':'average_ward_latitude'})
df = pd.merge(df,longitude_avg_by_lga,how="left",on="lga").rename(columns={'longitude':'average_lga_longitude','latitude':'average_lga_latitude'})

df_test = pd.merge(df_test,longitude_avg_by_region,how="left",on="region").rename(columns={'longitude_y':'average_region_longitude','latitude_y':'average_region_latitude'})
df_test = pd.merge(df_test,longitude_avg_by_district_code,how="left",on="district_code").rename(columns={'longitude':'average_district_code_longitude','latitude':'average_district_code_latitude'})
df_test = pd.merge(df_test,longitude_avg_by_ward,how="left",on="ward").rename(columns={'longitude':'average_ward_longitude','latitude':'average_ward_latitude'})
df_test = pd.merge(df_test,longitude_avg_by_lga,how="left",on="lga").rename(columns={'longitude':'average_lga_longitude','latitude':'average_lga_latitude'})

In [95]:
#add average height values to Dataframe
df = pd.merge(df,height_avg_by_region_complete,how="left",on="region").rename(columns={'gps_height_y':'average_region_height'})
df = pd.merge(df,height_avg_by_ward,how="left",on="ward").rename(columns={'gps_height':'average_ward_height'})
df = pd.merge(df,height_avg_by_district_code,how="left",on="district_code").rename(columns={'gps_height':'average_district_code_height'})
df = pd.merge(df,height_avg_by_lga,how="left",on="lga").rename(columns={'gps_height':'average_lga_height'})

df_test = pd.merge(df_test,height_avg_by_region_complete,how="left",on="region").rename(columns={'gps_height_y':'average_region_height'})
df_test = pd.merge(df_test,height_avg_by_ward,how="left",on="ward").rename(columns={'gps_height':'average_ward_height'})
df_test = pd.merge(df_test,height_avg_by_district_code,how="left",on="district_code").rename(columns={'gps_height':'average_district_code_height'})
df_test = pd.merge(df_test,height_avg_by_lga,how="left",on="lga").rename(columns={'gps_height':'average_lga_height'})

# Transformation Functions

In [96]:
#Classify longitude / height and construction years (zero values mean no data)
def classify_longitude(df):
    df['longitude_class'] = df['longitude_x'] != 0
    
def classify_gps_height(df):
    df['gps_height_class'] = df['gps_height_x'] != 0

def classify_construction_year(df):
    df['construction_year_class'] = df['construction_year'] !=0        

In [97]:
#Parse longitude and add to Dataframe
def add_parsed_longitude(df):
    if df['longitude_x']!=0:
        return df['longitude_x']
    elif not pd.isnull(df['average_ward_longitude']):
        return df['average_ward_longitude']
    elif not pd.isnull(df['average_lga_longitude']):
        return df['average_lga_longitude']
    elif not pd.isnull(df['average_district_code_longitude']):
        return df['average_district_code_longitude']
    else:
        return df['average_region_longitude']
    
def apply_parsed_longitude(df):
    df['parsed_longitude'] = df.apply(add_parsed_longitude, axis=1)        


In [98]:
#Parse latitude and add to Dataframe
def add_parsed_latitude(df):
    if df['latitude_x']!=0:
        return df['latitude_x']
    elif not pd.isnull(df['average_ward_latitude']):
        return df['average_ward_latitude']
    elif not pd.isnull(df['average_lga_latitude']):
        return df['average_lga_latitude']
    elif not pd.isnull(df['average_district_code_latitude']):
        return df['average_district_code_latitude']
    else:
        return df['average_region_latitude']
    
def apply_parsed_latitude(df):
    df['parsed_latitude'] = df.apply(add_parsed_latitude, axis=1)        

In [99]:
#convert values to integer values. Also as we have long float strings (e.g. latitude 35.51523) I'm going to multiply parsed loat / long values by 10000.
def convert_parsed_values_to_int64(df):
    df['parsed_height']=df['parsed_height'].astype('int64')
    df['parsed_longitude']=(df['parsed_longitude']).astype('int64')
    df['parsed_latitude']=(df['parsed_latitude']).astype('int64')
    df['amount_tsh']=df['amount_tsh'].astype('int64')
    df['district_code']=df['district_code'].astype('object')

In [101]:
#get rid of null values
def clear_null_values(df):
    df['funder_clean'] = df['funder'].fillna('unknown')
    df['installer_clean'] = df['installer'].fillna('unknown')
    df['subvillage_clean'] = df['subvillage'].fillna('none')
    df['public_meeting_clean'] = df['public_meeting'].fillna('none')
    df['scheme_management_clean'] = df['scheme_management'].fillna('none')
    df['scheme_name_clean'] = df['scheme_name'].fillna('none')
    df['permit_clean'] = df['permit'].fillna('unknown')
    df['construction_year_clean'] = df['construction_year'].fillna(0)
    df['installer'].replace(to_replace = '0', value ='unknown' , inplace=True) # filling 0 values with unknown

In [102]:
#Bin gps heights
def bin_gps(df):
    heightbins = [-100,-0.000001,0.00001,300,700,900,1100,1300,1500,1750,3000]
    heightlabels = ['below 0','zero','0<x<300','300<x<700','700<x<900','900<x<1100','1100<x<1300','1300<x<1500','1500<x<1750','1750<x<3000']
    df['gps_height_bins']=pd.cut(df['gps_height_x'], bins=heightbins, labels=heightlabels, include_lowest=True)

In [103]:
#Bin construction_year data
def bin_construction_year(df):
    constructionyearbins = [0,1959,1969,1979,1989,1999,2009,2022]
    constructionyearlabels = ['no record','60s','70s','80s','90s','00s','10s to 2022']
    df['construction_decade_bins']=pd.cut(df['construction_year_clean'], bins=constructionyearbins, labels=constructionyearlabels, include_lowest=True)

In [104]:
# Clean up 'installer' feature data
# Replacing the spelling mistakes and collect same categories in same name
def installer_cleanup(df):   
        df['installer_clean'] = df['installer_clean'].astype(str).str.lower()
        df['installer_clean'].replace(to_replace = ('district water department', 'district water depar','distric water department', 'district water department', 'district water depar', 'district council','district counci', 'village council orpha','kibaha town council','village council', 'coun', 'village counil', 'council','mbulu district council', 'counc', 'village council .oda','sangea district coun', 'songea district coun', 'villege council','district  council', 'quick win project /council', 'mbozi district council','village  council', 'municipal council', 'tabora municipal council','wb / district council','district water department', 'district water depar', 'district council','district counci', 'village council orpha','kibaha town council','village council', 'coun', 'village counil', 'council','mbulu district council', 'counc', 'village council .oda','sangea district coun', 'songea district coun', 'villege council','district  council', 'quick win project /council', 'mbozi district council','village  council', 'municipal council', 'tabora municipal council','wb / district council'),value ='district water council' , inplace=True)
        df['installer_clean'].replace(to_replace = ('commu', 'olgilai village community', 'adra /community', 'adra/community','rwe/ community', 'killflora /community', 'communit', 'taboma/community','arab community', 'adra/ community', 'sekei village community', 'rwe/community','arabs community', 'village community', 'government /community','dads/village community', 'killflora/ community', 'mtuwasa and community','rwe /community', 'ilwilo community', 'summit for water/community','igolola community', 'ngiresi village community', 'rwe community','african realief committe of ku', 'twesa /community', 'shelisheli commission','twesa/ community', 'marumbo community', 'government and community','community bank', 'kitiangare village community', 'oldadai village community','twesa/community', 'tlc/community', 'maseka community', 'islamic community','district community j', 'village water commission', 'village community members','tcrs/village community', 'village water committee', 'comunity'), value = 'community', inplace=True)
        df['installer_clean'].replace(to_replace = ('finw','fini water','fini water', 'fin water', 'finn water', 'finwater', 'finwate'), value ='fini water' , inplace=True)
        df['installer_clean'].replace(to_replace = ('coun', 'district council', 'district council','district Counci','district council','council','counc','district  council','distri'),value ='district council' , inplace=True)
        df['installer_clean'].replace(to_replace = ('rc church', 'rc churc', 'rc','rc ch','rc c', 'rc ch','rc church', 'rc cathoric', 'rc church', 'rc churc', 'rcchurch/cefa', 'irc', 'rc', 'rc ch', 'hw/rc','rc church/central gover', 'kkkt church', 'pentecost church', 'roman church','rc/mission', 'rc church/cefa', 'lutheran church', 'tag church','free pentecoste church of tanz', 'rc c', 'church', 'rc cathoric','morovian church', 'cefa/rc church', 'rc mission', 'anglican church','church of disciples', 'anglikana church', 'cetral government /rc','pentecostal church', 'cg/rc', 'rc missionary', 'sda church', 'methodist church', 'trc','rc msufi', 'haidomu lutheran church', 'baptist church', 'rc church brother','st magreth church', 'anglica church', 'global resource co', 'rc mi','baptist church of tanzania', 'fpct church', 'rc njoro', 'rc .church','rc mis', 'batist church', 'churc', 'dwe/anglican church','missi', 'mission','ndanda missions', 'rc/mission', 'cvs miss', 'missionaries', 'hydom luthelani','luthe', 'haydom lutheran hospital', 'lutheran', 'missio', 'germany missionary','grail mission kiseki bar', 'missionary', 'heri mission', 'german missionsry','wamissionari wa kikatoriki', 'neemia mission', 'wamisionari wa kikatoriki') , value ='rc church' , inplace=True)
        df['installer_clean'].replace(to_replace = ('central government','tanzania government','central government','cental government', 'cebtral government', 'tanzanian government','tanzania government', 'centra government' ,'central govt', 'centr', 'centra govt','central government', 'gove', 'central govt', 'gover', 'cipro/government','governme', 'adra /government', 'isf/government', 'adra/government','government /tcrs', 'village govt', 'government', 'government /community','concern /government', 'goverm', 'village government', 'cental government','govern', 'cebtral government', 'government /sda', 'tcrs /government','tanzania government', 'centra govt', 'colonial government', 'misri government','government and community', 'cetral government /rc', 'concern/government','government of misri', 'lwi &central government', 'governmen', 'government/tcrs', 'government /world vision','centra government') , value ='tanzania government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('world vision', 'world division','world vision','world vission', 'world division', 'word divisio','world visiin'),value ='world vision' , inplace=True)
        df['installer_clean'].replace(to_replace = ('unisef','unicef'),value ='unicef' , inplace=True)
        df['installer_clean'].replace(to_replace = 'danid', value ='danida' , inplace=True)
        df['installer_clean'].replace(to_replace = ('villigers', 'villager', 'villagers', 'villa', 'village', 'villi', 'village council','village counil', 'villages', 'vill', 'village community', 'villaers', 'village community', 'villag','villege council', 'village council','village  council','villagerd', 'villager', 'village technician','village office','village community members'),value ='villagers' , inplace=True)
        df['installer_clean'].replace(to_replace =('commu','communit','commu','commu', 'community') ,value ='community' , inplace=True)
        df['installer_clean'].replace(to_replace = ('government', 'gover', 'governme', 'goverm','govern','gover','gove','governme','governmen','go' ) ,value ='government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('government of misri') , value ='misri government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('italy government') , value ='italian government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('british colonial government') , value ='british government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('concern /government') , value ='concern/government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('village government') , value ='village government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('government and community') , value ='government /community' , inplace=True)
        df['installer_clean'].replace(to_replace = ('cetral government /rc') , value ='rc church/central gover' , inplace=True)
        df['installer_clean'].replace(to_replace = ('government /tcrs','government/tcrs') , value ='tcrs /government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('adra /government') , value ='adra/government' , inplace=True)
        df['installer_clean'].replace(to_replace = ('unicrf') , value ='unicef' , inplace=True)
        df['installer_clean'].replace(to_replace=('danid', 'danda','danida co', 'danny', 'daniad', 'dannida', 'danids'),value='danida', inplace=True)
        df['installer_clean'].replace(to_replace=('hesaws', 'huches', 'hesaw', 'hesawz', 'hesawq', 'hesewa'),value='hesawa', inplace=True)
        df['installer_clean'].replace(to_replace =('jaica','jaica co'), value ='jaica', inplace=True)
        df['installer_clean'].replace(to_replace=('dwsp', 'kkkt _ konde and dwe', 'rwe/dwe', 'rwedwe', 'dwe/', 'dw', 'dwr','dwe}', 'dwt', 'dwe /tassaf', 'dwe/ubalozi wa marekani', 'consultant and dwe','dwe & lwi', 'ubalozi wa marekani /dwe', 'dwe&', 'dwe/tassaf', 'dw$','dw e', 'tcrs/dwe', 'dw#', 'dweb', 'tcrs /dwe', 'water aid/dwe', 'dww'),value='dwe', inplace=True)
        df['installer_clean'].replace(to_replace=('africa muslim', 'muslimu society(shia)', 'africa muslim agenc','african muslims age', 'muslimehefen international','islamic','the isla', 'islamic agency tanzania',  'islam', 'nyabibuye islamic center'),value='muslims', inplace=True)
        df['installer_clean'].replace(to_replace=('british colonial government', 'british government', 'britain'),value='british', inplace=True)
        df['installer_clean'].replace(to_replace=('tcrs/tlc', 'tcrs /care', 'cipro/care/tcrs', 'tcrs kibondo', 'tcrs.tlc','tcrs /twesa', 'tassaf /tcrs', 'tcrs/care', 'tcrs twesa', 'rwe/tcrs','tcrs/twesa', 'tassaf/ tcrs', 'tcrs/ tassaf', 'tcrs/ twesa', 'tcrs a','tassaf/tcrs'),value='tcrs', inplace=True)
        df['installer_clean'].replace(to_replace=('kkkt-dioces ya pare', 'kkkt leguruki', 'kkkt ndrumangeni', 'kkkt dme','kkkt kilinga', 'kkkt canal', 'kkkt katiti juu', 'kkkt mareu'),value='kkkt', inplace=True)
        df['installer_clean'].replace(to_replace=('norad/'), value='norad', inplace=True)
        df['installer_clean'].replace(to_replace=('oxfarm'), value='oxfam', inplace=True)
        df['installer_clean'].replace(to_replace=('handeni trunk main('), value='handeni trunk main', inplace=True)
        df['installer_clean'].replace( to_replace=('tasaf/dmdd', 'dmdd/solider'),value='dmdd', inplace=True)
        df['installer_clean'].replace(to_replace=('cjejow construction', 'cjej0'), value='cjejow', inplace=True)
        df['installer_clean'].replace(to_replace=('china henan constuction', 'china henan contractor', 'china co.', 'chinese'),value='china', inplace=True)
        df['installer_clean'].replace(to_replace=('local contract', 'local technician', 'local', 'local  technician','locall technician', 'local te', 'local technitian', 'local technical tec','local fundi', 'local technical', 'localtechnician', 'village local contractor','local l technician'),value='local', inplace=True)
        df['installer_clean'].replace(to_replace=('oikos e .africa', 'oikos e.africa', 'africa amini alama','africa islamic agency tanzania', 'africare', 'african development foundation','oikos e. africa', 'oikos e.afrika', 'afroz ismail', 'africa', 'farm-africa','oikos e africa', 'farm africa', 'africaone', 'tina/africare', 'africaone ltd','african reflections foundation', 'africa m'),value='africa', inplace=True)
        df['installer_clean'].replace(to_replace=('0', 'nan', '-'), value='other', inplace=True)
        df_installer_count = df.groupby('installer_clean')['installer_clean'].count()
        to_other = df_installer_count[df_installer_count<71].index.tolist()
        df['installer_clean'].replace(to_replace = to_other, value = 'other', inplace=True)

        

In [105]:
#Clean up 'funder' feature data
# Replacing the spelling mistakes and collect same categories in same name
def funder_cleanup(df):
    # all lower case to ease matching
    df['funder_clean']=df['funder_clean'].astype(str).str.lower()
    # filling 0 and null values with unknown
    df['funder_clean'].fillna(value='Unknown',inplace=True)
    df['funder_clean'].replace(to_replace = '0', value ='Unknown' , inplace=True)
    df['funder_clean'].replace(to_replace=('kkkt_makwale', 'kkkt-dioces ya pare', 'world vision/ kkkt', 'kkkt church','kkkt leguruki', 'kkkt ndrumangeni', 'kkkt dme', 'kkkt canal', 'kkkt usa','kkkt mareu'),value='kkkt', inplace=True)
    df['funder'].replace(to_replace=('government of tanzania', 'norad /government', 'government/ community','cipro/government', 'isf/government', 'finidagermantanzania govt','government /tassaf', 'finida german tanzania govt', 'village government','tcrs /government', 'village govt', 'government/ world bank','danida /government', 'dhv/gove', 'concern /govern', 'vgovernment','lwi & central government', 'government /sda', 'koica and tanzania government','world bank/government', 'colonial government', 'misri government','government and community', 'concern/governm', 'government of misri','government/tassaf', 'government/school', 'government/tcrs', 'unhcr/government','government /world vision', 'norad/government'),value='government', inplace=True)
    df['funder'].replace(to_replace=('british colonial government', 'japan government', 'china government','finland government', 'belgian government', 'italy government','irish government', 'egypt government', 'iran gover', 'swedish', 'finland'),value='foreign government', inplace=True)
    df['funder'].replace(to_replace=('rc church', 'anglican church', 'rc churc', 'rc ch', 'rcchurch/cefa','irc', 'rc', 'churc', 'hw/rc', 'rc church/centr', 'pentecosta church','roman church', 'rc/mission', "ju-sarang church' and bugango",'lutheran church', 'roman cathoric church', 'tag church ub', 'aic church','free pentecoste church of tanz', 'tag church', 'fpct church', 'rc cathoric','baptist church', 'morovian church', 'cefa/rcchurch', 'rc mission','bukwang church saints', 'agt church', 'church of disciples', 'rc mofu',"gil cafe'church'", 'pentecostal church', 'bukwang church saint','eung am methodist church', 'rc/dwe', 'cg/rc', 'eung-am methodist church','rc missionary', 'sda church', 'methodist church', 'rc msufi','haidomu lutheran church', 'nazareth church', 'st magreth church','agape churc', 'rc missi', 'rc mi', 'rc njoro', 'world vision/rc church','pag church', 'batist church', 'full gospel church', 'nazalet church','dwe/anglican church', 'missi', 'mission', 'missionaries', 'cpps mission','cvs miss', 'grail mission kiseki bar', 'shelisheli commission', 'missionary','heri mission', 'german missionary', 'wamissionari wa kikatoriki','rc missionary', 'germany missionary', 'missio', 'neemia mission', 'rc missi','hydom luthelani', 'luthe', 'lutheran church',  'haydom lutheran hospital','village council/ haydom luther', 'lutheran', 'haidomu lutheran church','resolute golden pride project', 'resolute mininggolden pride','germany cristians'),value='church', inplace=True)
    df['funder'].replace(to_replace=('olgilai village community', 'commu', 'community', 'arab community','sekei village community', 'arabs community', 'village community','mtuwasa and community', 'ilwilo community', 'igolola community','ngiresi village community', 'marumbo community', 'village communi','comune di roma', 'comunity construction fund', 'community bank',"oak'zion' and bugango b' commu", 'kitiangare village community','oldadai village community', 'tlc/community', 'maseka community','islamic community',  'tcrs/village community', 'buluga subvillage community','okutu village community'),value='community', inplace=True)
    df['funder'].replace(to_replace=('council', 'wb / district council', 'cdtfdistrict council','sangea district council', 'mheza distric counc', 'kyela council','kibaha town council', 'swidish', 'mbozi district council', 'village council/ rose kawala',  'songea municipal counci','quick win project /council', 'village council', 'villege council','tabora municipal council', 'kilindi district co', 'kigoma municipal council','district council', 'municipal council', 'district medical','sengerema district council', 'town council', 'mkinga  distric cou','songea district council', 'district rural project', 'mkinga distric coun','dadis'),value='district', inplace=True)
    df['funder'].replace(to_replace=('tcrs.tlc', 'tcrs /care', 'tcrst', 'cipro/care/tcrs', 'tcrs/care', 'tcrs kibondo'),value='tcrs', inplace=True)
    df['funder'].replace(to_replace=('fini water', 'finw', 'fin water', 'finn water', 'finwater'),value='fini', inplace=True)
    df['funder'].replace(to_replace=('islamic', 'the isla', 'islamic found', 'islamic agency tanzania','islam', 'muislam', 'the islamic', 'nyabibuye islamic center', 'islamic society', 'african muslim agency','muslims', 'answeer muslim grou', 'muslimu society(shia)','unicef/african muslim agency', 'muslim world', 'muslimehefen international','shear muslim', 'muslim society'),value='islam', inplace=True)
    df['funder'].replace(to_replace=('danida', 'ms-danish', 'unhcr/danida', 'tassaf/ danida'),value='danida', inplace=True)
    df['funder'].replace(to_replace=('hesawa', 'hesawz', 'hesaw', 'hhesawa', 'hesawwa', 'hesawza', 'hesswa','hesawa and concern world wide'),value='hesawa', inplace=True)
    df['funder'].replace(to_replace=('world vision/adra', 'game division', 'worldvision'),value='world vision', inplace=True)
    df['funder'].replace(to_replace=('germany republi', 'a/co germany', 'aco/germany', 'bingo foundation germany','africa project ev germany', 'tree ways german'),value='germany', inplace=True)
    df['funder'].replace(to_replace=('0', 'nan', '-'), value='other', inplace=True)
    df_funder_cnt = df.groupby('funder')['funder'].count()
    other_list = df_funder_cnt[df_funder_cnt<100].index.tolist()
    df['funder'].replace(to_replace=other_list, value='other', inplace=True)        

In [106]:
#drop unneccessary fields
def drop_features(df):
    df.drop(['recorded_by','extraction_type_class','management_group',
             'payment_type','quality_group','quantity_group','source_type',
             'source_class','waterpoint_type_group','gps_height_x','longitude_x',
            'latitude_x','average_region_longitude','average_region_latitude','average_district_code_longitude',
            'average_district_code_latitude','average_ward_longitude','average_ward_latitude',
            'average_lga_longitude','average_lga_latitude','average_region_height','average_ward_height',
            'average_district_code_height','average_lga_height','funder','installer',
            'scheme_management','scheme_name','permit','construction_year','subvillage','date_recorded'], axis=1, inplace=True)

In [107]:

def get_medians_df(df):
    
    df_subvillage = df.groupby(['region_code'])['subvillage'].agg(pd.Series.mode)
    df_scheme = df.groupby(['region'])['scheme_name'].agg(pd.Series.mode)
    return df_subvillage, df_scheme
    

def scheme_restore(df, df_scheme):
    
    def scheme_update(row, df_scheme):
        row['scheme_name'] = df_scheme[row['region']]
        return row

    df.loc[df['scheme_name'].isnull(), ['scheme_name']] = \
        df[df['scheme_name'].isnull()].apply(
            scheme_update, df_scheme=df_scheme, axis=1)[['scheme_name']]
    
    
def subvillage_restore(df, df_subvillage):

    def subvillage_update(row, df_subvillage):
        row['subvillage'] = df_subvillage[row['region_code']]
        return row

    df.loc[df['subvillage'].isnull(), ['subvillage']] = \
        df[df['subvillage'].isnull()].apply(
            subvillage_update, df_subvillage=df_subvillage, axis=1)[['subvillage']]
    
    
def get_medians(df):
    
    df_pm_median = df['public_meeting'].median()
    df_permit_median = df['permit'].median()
    return df_pm_median, df_permit_median
    
    
def fill_na(df, df_pm_median, df_permit_median):
    
    df.loc[df['public_meeting'].isnull(), 'public_meeting'] = df_pm_median
    df.loc[df['permit'].isnull(), 'permit'] = df_permit_median
    
    
def create_na_features(df):    
    
    na_cols = ['subvillage', 'public_meeting', 'scheme_name', 'permit']
    for c in na_cols:
        df[f'c_na'] = df[c].isnull()  

In [109]:
#Apply Transformations to Training and Testing Datasets
set_classifier(df)
classify_longitude(df)
classify_gps_height(df)
classify_construction_year(df)    
apply_parsed_gps_height(df)
apply_parsed_longitude(df)
apply_parsed_latitude(df)
convert_parsed_values_to_int64(df)
clear_null_values(df)
bin_gps(df)
bin_construction_year(df)
installer_cleanup(df)
funder_cleanup(df)
df_subvillage, df_scheme = get_medians_df(df)
subvillage_restore(df, df_subvillage)
scheme_restore(df, df_scheme)
create_na_features(df)
df_pm_median, df_permit_median = get_medians(df)
fill_na(df, df_pm_median, df_permit_median)
drop_features(df)

#transform testing set
classify_longitude(df_test)
classify_gps_height(df_test)
classify_construction_year(df_test)    
apply_parsed_gps_height(df_test)
apply_parsed_longitude(df_test)
apply_parsed_latitude(df_test)
clear_null_values(df_test)
convert_parsed_values_to_int64(df_test)
bin_gps(df_test)
bin_construction_year(df_test)
installer_cleanup(df_test)
funder_cleanup(df_test)
df_test_subvillage, df_test_scheme = get_medians_df(df_test)
subvillage_restore(df_test, df_subvillage)
scheme_restore(df_test, df_scheme)
create_na_features(df_test)
df_pm_median, df_permit_median = get_medians(df_test)
fill_na(df_test, df_pm_median, df_permit_median)
drop_features(df_test)

In [110]:
#temporarily store our dataframes while we figure out model training
df2 = df
df2_test = df_test

In [ ]:
#snippet of code to restore our saved dataset if we need to!
#df=df2
#df_test=df2_test

In [111]:
df_train = df

In [ ]:
df_train.head()

# Build CatBoost Model

In [118]:
#Define our features (categorical & numeric)
num_features = ['amount_tsh', 'num_private','population','parsed_longitude','parsed_latitude','constuction_year_clean']
cat_features = [x for x in df.columns if x not in num_features and x!='status_group']

In [119]:
#Ensure all features have appropriate dtypes for CatBoost
df[cat_features] = df[cat_features].astype('category')
df_test[cat_features] = df_test[cat_features].astype('category')

In [120]:
#Define our Model fit function & Hyperparameters for CatBoost
def fit_model(train_pool, test_pool, **kwargs):
    model = CatBoostClassifier(
        max_ctr_complexity=5,
        task_type='CPU',
        iterations=10000,
        eval_metric='AUC',
        od_type='Iter',
        od_wait=500,
        **kwargs
    )

    return model.fit(
        train_pool,
        eval_set=test_pool,
        verbose=1000,
        plot=False,
        use_best_model=True)

In [121]:
#Define our classification assessment function
def classification_rate(y, y_pred):
    return np.sum(y==y_pred)/len(y)

In [122]:
#Perform Out of fold stratified cross validation
def get_oof(n_folds, x_train, y, x_test, cat_features, seeds):
    
    ntrain = x_train.shape[0]
    ntest = x_test.shape[0]  
        
    oof_train = np.zeros((len(seeds), ntrain, 3))
    oof_test = np.zeros((ntest, 3))
    oof_test_skf = np.empty((len(seeds), n_folds, ntest, 3))
    
    test_pool = Pool(data=x_test, cat_features=cat_features) 
    models = {}
    
    for iseed, seed in enumerate(seeds):
        kf = StratifiedKFold(
            n_splits=n_folds,
            shuffle=True,
            random_state=seed)          
        for i, (train_index, test_index) in enumerate(kf.split(x_train, y)):
            print(f'\nSeed {seed}, Fold {i}')
            x_tr = x_train.iloc[train_index, :]
            y_tr = y[train_index]
            x_te = x_train.iloc[test_index, :]
            y_te = y[test_index]
            train_pool = Pool(data=x_tr, label=y_tr, cat_features=cat_features)
            valid_pool = Pool(data=x_te, label=y_te, cat_features=cat_features) 

            model = fit_model(
                train_pool, valid_pool,
                loss_function='MultiClass',
                random_seed=seed
            )
            oof_train[iseed, test_index, :] = model.predict_proba(x_te)
            oof_test_skf[iseed, i, :, :] = model.predict_proba(x_test)
            models[(seed, i)] = model
            model.save_model(
                f"cb_{seed}_{i}_{VERSION}.cbm",
                format="cbm", export_parameters=None, pool=None)

    oof_test[:, :] = oof_test_skf.mean(axis=1).mean(axis=0)
    oof_train = oof_train.mean(axis=0)
    return oof_train, oof_test, models

In [ ]:
#train our model
y_train, X_train = df['status_group'], df.drop(['status_group'], axis=1)

oof_train, oof_test, models = get_oof(
    n_folds=FOLDS,
    x_train=X_train,
    y=y_train.values,
    x_test=df_test,
    cat_features=cat_features,
    seeds=SEEDS)


Seed 0, Fold 0
Learning rate set to 0.048559
0:	test: 0.7626007	best: 0.7626007 (0)	total: 317ms	remaining: 52m 47s
1000:	test: 0.9184141	best: 0.9184257 (999)	total: 4m 50s	remaining: 43m 35s
2000:	test: 0.9207637	best: 0.9209175 (1938)	total: 8m 47s	remaining: 35m 9s
3000:	test: 0.9224436	best: 0.9224496 (2999)	total: 12m 23s	remaining: 28m 55s
4000:	test: 0.9237064	best: 0.9237170 (3997)	total: 16m 1s	remaining: 24m 1s
5000:	test: 0.9242850	best: 0.9243097 (4789)	total: 19m 46s	remaining: 19m 46s
6000:	test: 0.9246647	best: 0.9247032 (5984)	total: 23m 39s	remaining: 15m 45s
7000:	test: 0.9252290	best: 0.9252317 (6997)	total: 27m 34s	remaining: 11m 48s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9253720411
bestIteration = 7129

Shrink model to first 7130 iterations.

Seed 0, Fold 1
Learning rate set to 0.048559
0:	test: 0.7799324	best: 0.7799324 (0)	total: 289ms	remaining: 48m 11s
1000:	test: 0.9304175	best: 0.9304594 (992)	total: 3m 46s	remaining: 33m 51s
2


Seed 42, Fold 4
Learning rate set to 0.048559
0:	test: 0.7689788	best: 0.7689788 (0)	total: 427ms	remaining: 1h 11m 8s
1000:	test: 0.9258844	best: 0.9258844 (1000)	total: 3m 58s	remaining: 35m 47s
2000:	test: 0.9277747	best: 0.9278192 (1994)	total: 8m 2s	remaining: 32m 7s
3000:	test: 0.9283931	best: 0.9285975 (2868)	total: 12m 10s	remaining: 28m 23s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9286391406
bestIteration = 3421

Shrink model to first 3422 iterations.

Seed 42, Fold 5
Learning rate set to 0.048559
0:	test: 0.7565810	best: 0.7565810 (0)	total: 312ms	remaining: 52m 4s
1000:	test: 0.9226362	best: 0.9226540 (994)	total: 3m 40s	remaining: 33m 1s
2000:	test: 0.9256923	best: 0.9256937 (1997)	total: 7m 18s	remaining: 29m 14s
3000:	test: 0.9271907	best: 0.9271954 (2987)	total: 10m 54s	remaining: 25m 26s
4000:	test: 0.9279526	best: 0.9279544 (3999)	total: 14m 33s	remaining: 21m 49s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.92811326


Seed 888, Fold 8
Learning rate set to 0.048559
0:	test: 0.7541962	best: 0.7541962 (0)	total: 223ms	remaining: 37m 13s
1000:	test: 0.9257788	best: 0.9257788 (1000)	total: 3m 15s	remaining: 29m 17s
2000:	test: 0.9286582	best: 0.9286641 (1991)	total: 6m 49s	remaining: 27m 18s
3000:	test: 0.9303161	best: 0.9303161 (3000)	total: 10m 19s	remaining: 24m 4s
4000:	test: 0.9310228	best: 0.9310504 (3940)	total: 13m 37s	remaining: 20m 25s
5000:	test: 0.9318288	best: 0.9318670 (4948)	total: 16m 58s	remaining: 16m 57s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9321685219
bestIteration = 5414

Shrink model to first 5415 iterations.

Seed 888, Fold 9
Learning rate set to 0.048559
0:	test: 0.7671993	best: 0.7671993 (0)	total: 290ms	remaining: 48m 23s
1000:	test: 0.9268607	best: 0.9268911 (989)	total: 3m 23s	remaining: 30m 28s
2000:	test: 0.9286727	best: 0.9287450 (1976)	total: 6m 43s	remaining: 26m 54s
3000:	test: 0.9295263	best: 0.9295343 (2998)	total: 9m 59s	remaining: 23m 


Seed 8888, Fold 2
Learning rate set to 0.048559
0:	test: 0.7815797	best: 0.7815797 (0)	total: 327ms	remaining: 54m 25s
1000:	test: 0.9247385	best: 0.9247647 (998)	total: 4m 21s	remaining: 39m 6s
2000:	test: 0.9274081	best: 0.9274140 (1999)	total: 8m 18s	remaining: 33m 11s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.9280773531
bestIteration = 2390

Shrink model to first 2391 iterations.

Seed 8888, Fold 3
Learning rate set to 0.048559
0:	test: 0.7718155	best: 0.7718155 (0)	total: 404ms	remaining: 1h 7m 15s
1000:	test: 0.9298885	best: 0.9299033 (994)	total: 3m 45s	remaining: 33m 48s
2000:	test: 0.9327781	best: 0.9327781 (2000)	total: 7m 12s	remaining: 28m 50s
3000:	test: 0.9340116	best: 0.9340116 (3000)	total: 10m 40s	remaining: 24m 53s
4000:	test: 0.9350112	best: 0.9350112 (4000)	total: 13m 55s	remaining: 20m 53s
5000:	test: 0.9353858	best: 0.9356152 (4901)	total: 17m 16s	remaining: 17m 15s
Stopped by overfitting detector  (500 iterations wait)

bestTest = 0.93

In [ ]:
#review feature importance of the model
m = models[(SEEDS[0],0)]
fea_imp = pd.DataFrame({'importance': m.feature_importances_,
                        'col': m.feature_names_})
fea_imp = fea_imp.sort_values(['importance', 'col'],
                               ascending=[True, False]).iloc[-40:]
fea_imp.plot(kind='barh', x='col', y='importance', figsize=(20, 20))

In [ ]:
#
y_pred_train = np.array([models[(SEEDS[0],0)].classes_[x] for x in oof_train.argmax(axis=1)])
print(f"balanced accuracy: {balanced_accuracy_score(y_train, y_pred_train)}")
class_rate = classification_rate(y_train, y_pred_train)
print(f"classification rate: {class_rate}")

In [ ]:
#
y_pred = np.array([models[(SEEDS[0],0)].classes_[x] for x in oof_test.argmax(axis=1)])
sub = pd.read_csv("SubmissionFormat.csv", index_col='id')
sub['status_group'] = y_pred
sub.to_csv(f"sub_{round(class_rate, 4)}.csv", index=True)